In [8]:
import openai
import json

from dotenv import load_dotenv
load_dotenv()

True

In [13]:
prompt1 = """
You are working as a content developer for an educational institution. You will be provided a CONTEXT_TEXT and number of questions to generate N, and your task will be to GENERATE Multiple Choice type Questions-Answers based on the provided text.

For every question, you will generate 4 choices for answer.

Below is the format that you must follow for your response.

```json
{
    "mcq1": {
        "q": <ques-text>,
        "c": [<choices for answer>],
        "a": <answer-text>
    }
}
```

You will only generate the json output, and nothing else. 


Example - 

CONTEXT_TEXT:
All water molecules form six-sided structures as they freeze and become snow 
crystals. The shape of the crystal is determined by temperature, vapor, and wind 
conditions in the upper atmosphere. Snow crystals are always symmetrical because 
these conditions affect all six sides simultaneously.

N: 1

QUESTIONS: 

{
    "mcqs": [
        {
        "q": "Where does the shape of crystal is determinded?",
        "c": ["Ground", "Space", "Upper Atmospere", "Jungle"],
        "a": "Upper Atmosphere"
        }
    ]
}

"""

prompt2 = """
You are working as a content developer for an educational institution. You will be provided a CONTEXT_TEXT and number of questions to generate N, and your task will be to GENERATE Multiple Choice type Questions-Answers based on the provided text.

For every question, you will generate 4 choices for answer.

Below is the format that you must follow for your response.

```json
{
    "mcq1": {
        "q": <ques-text>,
        "c": [<choices for answer>],
        "a": <answer-text>
    }
}
```

You will only generate the json output, and nothing else. 


Example - 

CONTEXT_TEXT:
All water molecules form six-sided structures as they freeze and become snow 
crystals. The shape of the crystal is determined by temperature, vapor, and wind 
conditions in the upper atmosphere. Snow crystals are always symmetrical because 
these conditions affect all six sides simultaneously.

N: 1

QUESTIONS: 

{
    "mcqs": [
        {
        "q": "The purpose of the passage is to present",
        "c": ["a personal observation.", "a solution to a problem.", "actual information.", "opposing scientific theories."],
        "a": "actual information."
        }
    ]
}

"""

system_message = {"role": "system", "content": "You are a help assistant"}
user_message = {"role": "user", "content": prompt2}


In [14]:
conversation = [system_message, user_message]

def generate_mcqs(conversation, text, model_id = "gpt-4"):

    input_prompt = f"""
    CONTEXT_TEXT:
    
    {text}
    
    N: 4
    
    QUESTIONS: 
    """

    input_message = {'role': 'user', "content": input_prompt}
    conversation.append(input_message)
    
    response = openai.ChatCompletion.create(
        model= model_id,
        messages = conversation,
        temperature = 0.2
    )

    output = response.choices[0].message.content

    return output

text = """In the words of Thomas DeQuincey, “It is notorious that the memory strengthens as 
you lay burdens upon it.” If, like most people, you have trouble recalling the names of
those you have just met, try this: The next time you are introduced, plan to remember 
the names. Say to yourself, “I’ll listen carefully; I’ll repeat each person’s name to be sure
I have it, and I will remember.” You’ll discover how effective this technique is and 
probably recall those names for the rest of your life"""

text = """
Many people have owned, or have heard of, traditional “piggy banks,” coin banks 
shaped like pigs. A logical theory about how this tradition started might be that because 
pigs often symbolize greed, the object is to “fatten” one’s piggy bank with as much 
money as possible. However, while this idea makes sense, it is not the correct
origin of the term. The genesis of the piggy bank is the old English word “pygg”, which 
was a common kind of clay hundreds of years ago in England. People used pots and 
jars made out of this red “pygg” clay for many different purposes in their homes. 
Sometimes they kept their money in one of the pots, and this was known as a pygg 
bank. Over the years, because “pygg” and “pig” sounded the same, glaziers began
making novelty banks out of pottery in the shape of a pig as a kind of joke. These banks 
were given as gifts and exported to countries where people spoke other languages and 
where no one had ever heard of pygg clay. The tradition caught on all over the world, 
and today piggy banks come in all colors and are made of all kinds of materials, 
including plastic.
"""

output = generate_mcqs(conversation, text)

print(output)


{
    "mcqs": [
        {
            "q": "What does the term 'piggy bank' originate from?",
            "c": ["The symbol of greed that pigs represent.", "The old English word 'pygg' which was a type of clay.", "The tradition of fattening pigs.", "The novelty of having a bank shaped like a pig."],
            "a": "The old English word 'pygg' which was a type of clay."
        },
        {
            "q": "Why did glaziers start making banks in the shape of pigs?",
            "c": ["Because pigs symbolize greed.", "Because 'pygg' and 'pig' sounded the same.", "Because pigs were considered lucky.", "Because they wanted to make a statement about saving money."],
            "a": "Because 'pygg' and 'pig' sounded the same."
        },
        {
            "q": "What were the original piggy banks made out of?",
            "c": ["Plastic.", "Glass.", "Pygg clay.", "Metal."],
            "a": "Pygg clay."
        },
        {
            "q": "How did the tradition of piggy banks spre

In [ ]:

{
    "mcqs": [
        {
        "q": "What does the old English word 'pygg' refer to?",
        "c": ["A type of pig", "A type of clay", "A type of bank", "A type of pot"],
        "a": "A type of clay"
        },
        {
        "q": "Why did glaziers start making banks in the shape of a pig?",
        "c": ["Because pigs symbolize greed", "Because 'pygg' and 'pig' sounded the same", "Because pigs were considered lucky", "Because pigs were common in England"],
        "a": "Because 'pygg' and 'pig' sounded the same"
        },
        {
        "q": "What was the original purpose of pygg banks?",
        "c": ["To store food", "To store money", "To store water", "To store clay"],
        "a": "To store money"
        },
        {
        "q": "What are modern piggy banks made of?",
        "c": ["Only clay", "Only plastic", "Only metal", "All kinds of materials"],
        "a": "All kinds of materials"
        }
    ]
}

In [9]:
json_obj = json.loads(output)

json_obj

{'mcqs': [{'q': "Who said 'It is notorious that the memory strengthens as you lay burdens upon it.'?",
   'c': ['Albert Einstein',
    'Thomas DeQuincey',
    'William Shakespeare',
    'Isaac Newton'],
   'a': 'Thomas DeQuincey'},
  {'q': 'What is the suggested technique to remember names?',
   'c': ['Writing them down',
    'Repeating the name',
    'Taking a picture',
    'None of the above'],
   'a': 'Repeating the name'},
  {'q': 'What is the common problem most people face according to the text?',
   'c': ['Forgetting faces',
    'Forgetting names',
    'Forgetting places',
    'Forgetting events'],
   'a': 'Forgetting names'},
  {'q': 'What is the effect of the suggested technique according to the text?',
   'c': ['It is not effective',
    'It is somewhat effective',
    'It is very effective',
    'The text does not mention the effect'],
   'a': 'It is very effective'}]}